## Extract Hotels info

#### Extractors

In [32]:
from bs4 import BeautifulSoup
import re
from typing import Dict, Optional, Union

def extract_price(hotel: BeautifulSoup) -> Dict[str, Optional[Union[int, float]]]:
    discounted_price: Optional[Union[int, float]] = None
    original_price: Optional[Union[int, float]] = None

    # Extract discounted price: Look for a div with either "uitk-type-500" or "ausitk-type-500"
    price_elem = hotel.find("div", class_=lambda x: x and ("uitk-type-500" in x or "ausitk-type-500" in x))
    if price_elem:
        price_text = price_elem.get_text(strip=True)
        price_digits = re.sub(r'[^\d.]', '', price_text)
        try:
            if '.' in price_digits:
                discounted_price = float(price_digits)
            else:
                discounted_price = int(price_digits)
        except ValueError:
            discounted_price = None

    # Extract original price from <del> element (if present)
    del_elem = hotel.find("del")
    if del_elem:
        op_text = del_elem.get_text(strip=True)
        op_digits = re.sub(r'[^\d.]', '', op_text)
        try:
            if '.' in op_digits:
                original_price = float(op_digits)
            else:
                original_price = int(op_digits)
        except ValueError:
            original_price = None

    # If original price is missing, default it to the discounted price.
    if original_price is None and discounted_price is not None:
        original_price = discounted_price

    return {
        "discounted_price": discounted_price,
        "original_price": original_price
    }


In [33]:
from typing import Dict, Optional, Union

def extract_review_info(hotel: BeautifulSoup) -> Dict[str, Optional[Union[float, int, str]]]:
    review_score: Optional[float] = None
    review_title: Optional[str] = None
    number_of_reviews: Optional[int] = None

    # Extract review score from a badge element (e.g., <span> with class "uitk-badge-base-text")
    review_badge = hotel.find("span", class_=lambda x: x and "uitk-badge-base-text" in x)
    if review_badge:
        rs_text = review_badge.get_text(strip=True)
        rs_clean = re.sub(r'[^\d.]', '', rs_text)
        try:
            review_score = float(rs_clean)
        except ValueError:
            review_score = None

    # Extract review title (searching for common descriptors)
    review_title_elem = hotel.find("span", string=re.compile(r"(Excellent|Very good|Good)", re.IGNORECASE))
    if review_title_elem:
        review_title = review_title_elem.get_text(strip=True)

    # Extract number of reviews (e.g., text like "1,415 reviews")
    reviews_elem = hotel.find("span", string=re.compile(r"reviews", re.IGNORECASE))
    if reviews_elem:
        rev_text = reviews_elem.get_text(strip=True)
        rev_digits = re.sub(r'\D', '', rev_text)
        try:
            number_of_reviews = int(rev_digits)
        except ValueError:
            number_of_reviews = None

    return {
        "review_score": review_score,
        "review_title": review_title,
        "number_of_reviews": number_of_reviews,
    }

In [34]:
from typing import Optional
import re
from bs4 import BeautifulSoup

def extract_rating(hotel: BeautifulSoup) -> Dict[str, Optional[float]]:
    star_rating: Optional[float] = None
    rating_div = hotel.find("div", class_=lambda x: x and "uitk-rating" in x)
    if rating_div:
        hidden_rating = rating_div.find("span", class_="is-visually-hidden")
        if hidden_rating:
            rating_text = hidden_rating.get_text(strip=True)
            match = re.search(r"(\d+(\.\d+)?)", rating_text)
            if match:
                try:
                    star_rating = float(match.group(1))
                except ValueError:
                    star_rating = None
    return {
        "star_rating": star_rating
        }


In [35]:
from typing import Dict, Optional
from bs4 import BeautifulSoup

def extract_neighborhood(hotel: BeautifulSoup) -> Dict[str, Optional[str]]:
    neighborhood_elem = hotel.find("div", class_=lambda x: x and "truncate-lines-2" in x)
    neighborhood = neighborhood_elem.get_text(strip=True) if neighborhood_elem else None
    return {
        "neighborhood": neighborhood
        }


In [36]:
from typing import Dict, Optional
import re
from bs4 import BeautifulSoup

def extract_booking_options(hotel: BeautifulSoup) -> Dict[str, Optional[bool]]:
    breakfast_included: bool = False
    free_cancellation: bool = False
    prepayment_needed: Optional[bool] = None

    # Check for breakfast included by looking for tags containing both "breakfast" and "included"
    breakfast_elem = hotel.find(lambda tag: tag.name in ['div', 'span'] and 
                                  "breakfast" in tag.get_text(strip=True).lower() and 
                                  "included" in tag.get_text(strip=True).lower())
    if breakfast_elem:
        breakfast_included = True

    # Check for free cancellation: look for "fully refundable" anywhere in the text
    if hotel.find(string=re.compile(r"fully refundable", re.IGNORECASE)):
        free_cancellation = True

    # Determine prepayment needed:
    # If "reserve now, pay later" is found then prepayment is not needed (False).
    # If "prepayment required" or "prepayment needed" is found then prepayment is needed (True).
    if hotel.find(string=re.compile(r"reserve now, pay later", re.IGNORECASE)):
        prepayment_needed = False
    elif hotel.find(string=re.compile(r"prepayment (required|needed)", re.IGNORECASE)):
        prepayment_needed = True

    return {
        "breakfast_included": breakfast_included,
        "free_cancellation": free_cancellation,
        "prepayment_needed": prepayment_needed,
    }


In [37]:
from typing import Dict, Optional
from bs4 import BeautifulSoup

def extract_name(hotel: BeautifulSoup) -> Dict[str, Optional[str]]:
    name_elem = hotel.find("h3", class_=lambda x: x and "uitk-heading" in x)
    name = name_elem.get_text(strip=True) if name_elem else None
    return {"name": name}


#### Aggragator

In [38]:
def extract_hotel_info(hotel: BeautifulSoup) -> Dict[str, Optional[Union[str, int, float, bool]]]:
    extractors = [
        extract_name,
        extract_price,
        extract_rating,
        extract_review_info,
        extract_neighborhood,
        extract_booking_options,
    ]
    
    # Merge all dictionaries returned by each extractor function using dictionary unpacking.
    return {**{k: v for extractor in extractors for k, v in extractor(hotel).items()}}

In [39]:
# --- Example usage ---
html_doc = """
<div class="uitk-spacing uitk-spacing-margin-blockstart-three"><div><div class="uitk-card uitk-card-roundcorner-all uitk-card-has-border uitk-card-has-primary-theme" data-stid="lodging-card-responsive"><div class="uitk-layout-grid uitk-layout-grid-has-auto-columns uitk-layout-grid-has-columns-by-medium uitk-layout-grid-display-grid" style="--uitk-layoutgrid-auto-columns: minmax(var(--uitk-layoutgrid-egds-size__0x), 1fr); --uitk-layoutgrid-columns-medium: repeat(3, minmax(0, 1fr));"><div class="LazyLoad is-visible"><div class="uitk-layout-flex uitk-layout-flex-block-size-full-size uitk-layout-position uitk-layout-position-top-zero uitk-layout-position-left-zero uitk-layout-position-relative" data-stid="lodging-card-media-section"><figure class="uitk-image uitk-layout-position uitk-layout-position-zindex-layer2 uitk-image-fit-cover uitk-image-ratio-21-9 uitk-image-ratio"><span><div class="uitk-gallery-carousel"><h3 class="is-visually-hidden">Photo gallery for Ameritania Hotel at Times Square</h3><div class="uitk-gallery-carousel-items ratio-21-9"><div class="uitk-gallery-carousel-item uitk-gallery-carousel-item-prev" aria-hidden="true"><figure class="uitk-image uitk-image-fit-cover uitk-image-ratio-21-9 uitk-image-ratio"><div class="uitk-image-placeholder"><img alt="Room" class="uitk-image-media" src="https://images.trvl-media.com/lodging/1000000/30000/20900/20842/79fd23fe.jpg?impolicy=resizecrop&amp;rw=455&amp;ra=fit" data-loaded="true"></div></figure></div><div class="uitk-gallery-carousel-item uitk-gallery-carousel-item-current" aria-hidden="false"><figure class="uitk-image uitk-image-fit-cover uitk-image-ratio-21-9 uitk-image-ratio"><div class="uitk-image-placeholder"><img alt="View from room" class="uitk-image-media" src="https://images.trvl-media.com/lodging/1000000/30000/20900/20842/01603f7f.jpg?impolicy=resizecrop&amp;rw=455&amp;ra=fit"></div><button type="button" data-testid="uitk-gallery-item-current-trigger" aria-label="More information about Ameritania Hotel at Times Square, opens in a new tab" class="uitk-image-link"><span class="is-visually-hidden">View from room</span></button></figure></div><div class="uitk-gallery-carousel-item uitk-gallery-carousel-item-next" aria-hidden="true"><figure class="uitk-image uitk-image-fit-cover uitk-image-ratio-21-9 uitk-image-ratio"><div class="uitk-image-placeholder"><img alt="Front of property" class="uitk-image-media" src="https://images.trvl-media.com/lodging/1000000/30000/20900/20842/a8ca261d.jpg?impolicy=resizecrop&amp;rw=455&amp;ra=fit" data-loaded="true"></div></figure></div><div class="uitk-gallery-carousel-item uitk-gallery-carousel-item-queued" aria-hidden="true"><figure class="uitk-image uitk-image-fit-cover uitk-image-ratio-21-9 uitk-image-ratio"><div class="uitk-image-placeholder"><img alt="Room" class="uitk-image-media" src="https://images.trvl-media.com/lodging/1000000/30000/20900/20842/d84b060e.jpg?impolicy=resizecrop&amp;rw=455&amp;ra=fit" data-loaded="true"></div></figure></div></div><div class="uitk-gallery-carousel-paging-controls"><button theme="transparent-btn" type="button" class="uitk-button uitk-button-medium uitk-button-only-icon uitk-gallery-carousel-button-prev uitk-button-paging uitk-button-paging-overlay"><svg class="uitk-icon uitk-icon-leading uitk-icon-directional" aria-label="Show previous image for Ameritania Hotel at Times Square" role="img" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><title id="prev-button-lodging-card-gallery-carousel-20842-[object Object]-title">Show previous image for Ameritania Hotel at Times Square</title><path d="M14.862 16.448 10.414 12l4.446-4.447a.5.5 0 0 0-.007-.7l-.707-.707a.5.5 0 0 0-.707 0l-5.146 5.147a1 1 0 0 0 0 1.414l5.146 5.146a.5.5 0 0 0 .707 0l.707-.707a.5.5 0 0 0 .009-.698z"></path></svg></button><button theme="transparent-btn" type="button" class="uitk-button uitk-button-medium uitk-button-only-icon uitk-gallery-carousel-button-next uitk-button-paging uitk-button-paging-overlay"><svg class="uitk-icon uitk-icon-leading uitk-icon-directional" aria-label="Show next image for Ameritania Hotel at Times Square" role="img" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><title id="next-button-lodging-card-gallery-carousel-20842-[object Object]-title">Show next image for Ameritania Hotel at Times Square</title><path d="M10.56 6.146a.5.5 0 0 0-.706 0l-.708.708a.5.5 0 0 0 0 .707L13.586 12l-4.44 4.44a.5.5 0 0 0 0 .706l.708.708a.5.5 0 0 0 .707 0l5.146-5.147a1 1 0 0 0 0-1.414l-5.146-5.147z"></path></svg></button></div></div></span></figure><div class="uitk-layout-position uitk-layout-position-top-zero uitk-layout-position-right-zero uitk-layout-position-zindex-layer2 uitk-layout-position-absolute"> <div data-stid="outlined-save-button" id="trip-save-item-cqz"><div class="favorite-button-wrapper"><button type="button" aria-label="Sign in to save Ameritania Hotel at Times Square to a trip" class="favorite-button favorite-button-button-no-label"><svg class="uitk-icon favorite-button-fill favorite-button-fill-default" aria-hidden="true" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path d="M22 8.688c-.043-4.41-4.738-7.196-8.644-4.817A6.39 6.39 0 0 0 12 5c-.365-.4-.82-.815-1.356-1.142C6.82 1.524 1.977 4.24 2 8.688c.015 2.873 1.4 5.457 3.365 7.498 1.797 1.865 4.056 3.397 6.117 4.668a1 1 0 0 0 1.052-.001c2.04-1.266 4.312-2.81 6.12-4.682 1.98-2.05 3.373-4.639 3.346-7.483z"></path></svg><svg class="uitk-icon favorite-button-border favorite-button-border-default favorite-button-border-color-highlighed" aria-hidden="true" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path fill-rule="evenodd" d="M13.356 3.871c3.906-2.379 8.601.407 8.644 4.817.027 2.844-1.367 5.433-3.346 7.483-1.808 1.873-4.08 3.416-6.12 4.682a1 1 0 0 1-1.052.001c-2.061-1.271-4.32-2.803-6.117-4.668C3.399 14.146 2.015 11.56 2 8.688c-.023-4.448 4.819-7.164 8.644-4.83.537.327.991.743 1.356 1.142a6.39 6.39 0 0 1 1.356-1.129zM20 8.708c-.028-2.912-3.067-4.673-5.604-3.128-.408.248-.754.602-1.151 1.065a1 1 0 0 1-.76.349h-.961a1 1 0 0 1-.76-.35c-.401-.47-.75-.828-1.162-1.08C7.098 4.038 3.985 5.815 4 8.679c.011 2.214 1.081 4.33 2.805 6.12 1.477 1.533 3.343 2.855 5.2 4.026 1.844-1.169 3.722-2.502 5.21-4.042C18.96 12.975 20.02 10.865 20 8.708z" clip-rule="evenodd"></path></svg></button></div></div> </div></div></div><div class="uitk-layout-position uitk-layout-position-relative uitk-layout-grid-item uitk-layout-grid-item-has-column-start-by-medium" style="--uitk-layoutgrid-column-start-medium: span 2;"> <div class="uitk-card-content-section uitk-card-content-section-padded"><div class="uitk-layout-flex uitk-layout-flex-block-size-full-size uitk-layout-flex-flex-direction-column uitk-layout-flex-justify-content-space-between" style="min-height: 143px;"><div class="uitk-spacing uitk-spacing-padding-blockend-three uitk-layout-flex-item"><div class="uitk-layout-flex uitk-layout-flex-justify-content-space-between uitk-layout-flex-gap-two"><div class="uitk-layout-grid uitk-layout-grid-has-auto-columns uitk-layout-grid-has-rows uitk-layout-grid-display-grid uitk-layout-flex-item" style="--uitk-layoutgrid-auto-columns: minmax(var(--uitk-layoutgrid-egds-size__0x), 1fr); --uitk-layoutgrid-rows: min-content;"><h3 class="uitk-heading uitk-heading-5 overflow-wrap uitk-layout-grid-item uitk-layout-grid-item-has-row-start" style="--uitk-layoutgrid-row-start: span 1;">Ameritania Hotel at Times Square</h3><div class="uitk-rating"><span class="is-visually-hidden">3.5 out of 5</span><svg class="uitk-icon uitk-rating-icon uitk-icon-xsmall" aria-hidden="true" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path d="M12.648 2.478a.678.678 0 0 0-1.296 0L9.333 9h-6.68a.653.653 0 0 0-.365 1.194l5.428 3.657-1.978 6.38a.593.593 0 0 0 .919.653L12 16.944l5.343 3.94a.593.593 0 0 0 .919-.653l-1.978-6.38 5.428-3.657A.653.653 0 0 0 21.347 9h-6.68l-2.02-6.522z"></path></svg><svg class="uitk-icon uitk-rating-icon uitk-icon-xsmall" aria-hidden="true" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path d="M12.648 2.478a.678.678 0 0 0-1.296 0L9.333 9h-6.68a.653.653 0 0 0-.365 1.194l5.428 3.657-1.978 6.38a.593.593 0 0 0 .919.653L12 16.944l5.343 3.94a.593.593 0 0 0 .919-.653l-1.978-6.38 5.428-3.657A.653.653 0 0 0 21.347 9h-6.68l-2.02-6.522z"></path></svg><svg class="uitk-icon uitk-rating-icon uitk-icon-xsmall" aria-hidden="true" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path d="M12.648 2.478a.678.678 0 0 0-1.296 0L9.333 9h-6.68a.653.653 0 0 0-.365 1.194l5.428 3.657-1.978 6.38a.593.593 0 0 0 .919.653L12 16.944l5.343 3.94a.593.593 0 0 0 .919-.653l-1.978-6.38 5.428-3.657A.653.653 0 0 0 21.347 9h-6.68l-2.02-6.522z"></path></svg><svg class="uitk-icon uitk-rating-icon uitk-icon-directional uitk-icon-xsmall" aria-hidden="true" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path fill-rule="evenodd" d="M12 2a.678.678 0 0 0-.648.478L9.333 9h-6.68a.653.653 0 0 0-.365 1.194l5.428 3.657-1.978 6.38a.593.593 0 0 0 .919.653L12 16.944V2z" clip-rule="evenodd"></path></svg></div><div class="uitk-text uitk-text-spacing-half truncate-lines-2 uitk-type-300 uitk-text-default-theme" aria-hidden="false">Theater District</div></div></div></div><div class="uitk-layout-grid uitk-layout-grid-has-auto-columns uitk-layout-grid-has-columns uitk-layout-grid-has-columns-by-medium uitk-layout-grid-has-columns-by-large uitk-layout-grid-has-space uitk-layout-grid-display-grid uitk-layout-flex-item" style="--uitk-layoutgrid-auto-columns: minmax(var(--uitk-layoutgrid-egds-size__0x), 1fr); --uitk-layoutgrid-columns: 1fr 2fr 2fr; --uitk-layoutgrid-columns-medium: 1fr 2fr 2fr; --uitk-layoutgrid-columns-large: 1fr max-content max-content; --uitk-layoutgrid-column-gap: var(--uitk-layoutgrid-space-one); --uitk-layoutgrid-row-gap: var(--uitk-layoutgrid-space-one);"><div class="uitk-layout-grid-item uitk-layout-grid-item-align-self-end uitk-layout-grid-item-has-column-start" style="--uitk-layoutgrid-column-start: span 2;"><div class="uitk-layout-flex uitk-layout-flex-flex-direction-column uitk-layout-flex-gap-three uitk-spacing uitk-spacing-margin-blockstart-"><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-layout-flex-gap-one"><div class="uitk-layout-flex uitk-layout-flex-align-items-center"><span class="uitk-badge uitk-badge-base-large uitk-badge-base-has-text uitk-badge-positive"><span class="uitk-badge-base-text" aria-hidden="true">8.8</span><span class="is-visually-hidden">8.8 out of 10</span></span></div><div class="uitk-layout-flex uitk-layout-flex-flex-direction-column uitk-layout-flex-item"><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-layout-flex-gap-one"><span class="uitk-text uitk-type-300 uitk-type-medium uitk-text-emphasis-theme" aria-hidden="true">Excellent</span><span class="is-visually-hidden">excellent</span></div><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-layout-flex-gap-one"><span class="uitk-text uitk-type-200 uitk-type-regular uitk-text-default-theme" aria-hidden="true">3,783 reviews</span><span class="is-visually-hidden">(3,783 reviews)</span></div></div></div></div></div><div class="uitk-layout-flex uitk-layout-flex-flex-direction-column uitk-layout-grid-item uitk-layout-grid-item-align-self-end uitk-layout-grid-item-has-column-start uitk-layout-grid-item-justify-self-end" style="--uitk-layoutgrid-column-start: span 1;"><div class="uitk-layout-flex uitk-layout-flex-flex-direction-column" data-test-id="price-summary"><div><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-layout-flex-flex-direction-row uitk-layout-flex-justify-content-flex-end uitk-layout-flex-gap-xsmall uitk-layout-flex-flex-wrap-wrap" data-test-id="price-summary-message-line"><button type="button" class="neutral-color-link neutral-color-link-align-left neutral-color-link-layout-inline neutral-color-link-medium" data-stid="disclaimer-dialog-link"><div class="uitk-layout-flex uitk-layout-flex-flex-direction-row"><div class="uitk-spacing uitk-spacing-padding-blockstart-half uitk-layout-flex-item"><div class="uitk-text uitk-type-300 uitk-text-default-theme is-visually-hidden">The previous price was €454</div><span aria-hidden="true"><div class="uitk-text uitk-type-300 uitk-text-default-theme"><del>€454</del></div></span></div></div></button><div class="uitk-layout-position uitk-layout-position-relative uitk-spacing uitk-spacing-padding-blockstart-half"> <div><div class="uitk-text uitk-type-300 uitk-text-default-theme is-visually-hidden">The current price is €433</div><span aria-hidden="true"><div class="uitk-text uitk-type-500 uitk-type-medium uitk-text-emphasis-theme">€433</div></span></div> </div></div><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-layout-flex-flex-direction-row uitk-layout-flex-justify-content-flex-end uitk-layout-flex-gap-xsmall uitk-layout-flex-flex-wrap-wrap" data-test-id="price-summary-message-line"><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-spacing uitk-spacing-margin-blockstart-unset"><div><div class="uitk-text uitk-type-end uitk-type-200 uitk-text-default-theme">for 1 room</div></div></div></div><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-layout-flex-flex-direction-row uitk-layout-flex-justify-content-flex-end uitk-layout-flex-gap-xsmall uitk-layout-flex-flex-wrap-wrap" data-test-id="price-summary-message-line"><div class="uitk-layout-flex uitk-layout-flex-align-items-center uitk-spacing uitk-spacing-margin-blockstart-unset"><div><div class="uitk-text uitk-type-end uitk-type-200 uitk-text-default-theme">includes taxes &amp; fees</div></div></div></div></div></div><div class="uitk-spacing uitk-spacing-margin-blockstart-two uitk-layout-position uitk-layout-position-zindex-layer2 uitk-layout-flex-item-align-self-flex-end uitk-layout-flex-item"><div><a type="button" href="https://euro.expedia.net/user/signin?uurl=e3id%3Dredr%26rurl%3D%2FHotel-Search%3FregionId%3D2621%26destination%3DNew%2BYork%252C%2BNew%2BYork%252C%2BUnited%2BStates%2Bof%2BAmerica%26startDate%3D2025-03-18%26endDate%3D2025-03-19%26adults%3D2%26userIntent%3D%26sort%3DRECOMMENDED%26vip%3Dfalse%26selected%3D20842&amp;partnerLang=en_IE" class="uitk-button uitk-button-small uitk-button-has-text uitk-button-as-link uitk-button-primary uitk-layout-flex-item-align-self-flex-end uitk-layout-flex-item"><svg class="uitk-icon uitk-icon-leading" aria-hidden="true" viewBox="0 0 24 24" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink"><path fill-rule="evenodd" d="m17.1 16.05 1.22-1.18 3.48-3.48a.78.78 0 0 0 .2-.6l-1.3-7.1a.57.57 0 0 0-.42-.42l-7.06-1.26a.78.78 0 0 0-.61.19L2.1 12.7a.36.36 0 0 0 0 .51l8.68 8.69c.14.13.37.14.5 0l4.24-4.23a2.88 2.88 0 0 0 4.9-2.26v-.1c0-.28-.03-1.02-.26-1.5L19 14.9a1.54 1.54 0 1 1-3 .62c-.03-.5.19-2.34.5-4.07a26.11 26.11 0 0 1 .56-2.48l.02.04a1.62 1.62 0 1 0-1.42-.12c-.13.57-.26 1.26-.41 2.1a29.62 29.62 0 0 0-.57 4.88l-.83.83-6.56-6.55 6.04-6.04c.07-.08.2-.12.3-.1l5.2.94c.09.01.18.1.2.2l.98 5.18c.02.1-.03.23-.1.3l-3.1 3.1c-.2.75-.43 2.05.3 2.31zm-6.24 3.4-6.29-6.32a.18.18 0 0 1 0-.25l1.72-1.72 6.56 6.56-1.74 1.73a.18.18 0 0 1-.25 0z" clip-rule="evenodd"></path></svg>Sign in for extra savings</a></div></div><div class="uitk-layout-flex-item-align-self-flex-end uitk-layout-flex-item"></div></div></div></div></div> </div></div><a rel="noopener" data-stid="open-hotel-information" dd-action-name="open-hotel-information" target="_blank" href="/New-York-Hotels-Ameritania-At-Times-Square.h20842.Hotel-Information?chkin=2025-03-18&amp;chkout=2025-03-19&amp;x_pwa=1&amp;rfrr=HSR&amp;pwa_ts=1741121108522&amp;referrerUrl=aHR0cHM6Ly9ldXJvLmV4cGVkaWEubmV0L0hvdGVsLVNlYXJjaA%3D%3D&amp;useRewards=false&amp;rm1=a2&amp;regionId=2621&amp;destination=New+York%2C+New+York%2C+United+States+of+America&amp;destType=MARKET&amp;neighborhoodId=6157019&amp;latLong=40.712843%2C-74.005966&amp;sort=RECOMMENDED&amp;top_dp=433&amp;top_cur=EUR&amp;gclid=CjwKCAiA5pq-BhBuEiwAvkzVZTW4ZLUmdKrp8Zc0eio50l9G-JFf6oMj3ADjgjMR8ME4AR-cPr2_JRoCZXsQAvD_BwE&amp;semcid=EU.B.GOOGLE.BD-c-EN.HOTEL&amp;semdtl=a118946203262.b1147109973514.g1aud-2191897288246%3Akwd-864305033.e1c.m1CjwKCAiA5pq-BhBuEiwAvkzVZTW4ZLUmdKrp8Zc0eio50l9G-JFf6oMj3ADjgjMR8ME4AR-cPr2_JRoCZXsQAvD_BwE.r107063433bf2c9e90cb9a76256c8d4d3c30b9f11c1563796c64a22ed04b078828.c1Q-aBm595YVEj9mSfdsmmEA.j11008005.k1.d1640928270367.h1e.i1.l1.n1.o1.p1.q1.s1.t1.x1.f1.u1.v1.w1&amp;semdtl=a118946203262.b1147109973514.g1aud-2191897288246%3Akwd-864305033.e1c.m1CjwKCAiA5pq-BhBuEiwAvkzVZTW4ZLUmdKrp8Zc0eio50l9G-JFf6oMj3ADjgjMR8ME4AR-cPr2_JRoCZXsQAvD_BwE.r107063433bf2c9e90cb9a76256c8d4d3c30b9f11c1563796c64a22ed04b078828.c1Q-aBm595YVEj9mSfdsmmEA.j11008005.k1.d1640928270367.h1e.i1.l1.n1.o1.p1.q1.s1.t1.x1.f1.u1.v1.w1&amp;userIntent=&amp;selectedRoomType=321638776&amp;selectedRatePlan=389193577&amp;searchId=7152a0bd-14e0-4a69-a677-42fdd2a3fed2" class="uitk-card-link"><span class="is-visually-hidden">More information about Ameritania Hotel at Times Square, opens in a new tab</span></a></div></div></div>
"""

# Run the scraping function
hotel_data = extract_hotel_info(BeautifulSoup(html_doc, "html.parser"))
print(hotel_data)


{'name': 'Ameritania Hotel at Times Square', 'discounted_price': 433, 'original_price': 454, 'star_rating': 3.5, 'review_score': 8.8, 'review_title': 'Excellent', 'number_of_reviews': 3783, 'neighborhood': 'Theater District', 'breakfast_included': False, 'free_cancellation': False, 'prepayment_needed': None}
